In [1]:
import pandas as pd

df = pd.read_csv('lang_rna.csv')
df.head()
row, col = df.shape

In [3]:
# prepare for running hh-suite (see utils/runHHblits.py)
for i in range(row):
    seq_1 = df.iloc[i,2]
    seq_2 = df.iloc[i,4]
    entry_id = df.iloc[i,0]
    # label = fasta.readline().strip()
    # print(i)
    new_1 = open("fasta/pro_mut_" + str(entry_id) + ".fasta", "w+")
    new_2 = open("fasta/pro_ori_" + str(entry_id) + ".fasta", "w+")
    new_1.write(">" + str(entry_id) + '\n')
    new_1.write(seq_1)
    new_2.write(">" + str(entry_id) + '\n')
    new_2.write(seq_2)
    if i == (row-1):
        print(seq_1,seq_2)
    # line = fasta.readline()

MASFHLDHLKNYHRRFCRPSRAPNIPTKQGQNMLEILQDCFEEQNSTESLSCSTPKEKDIYVQSTGKERQASHSESVPISSRRKEASLQVPAKPSDPADEPVQANEVHQKILSTDVVSKSTPDSTKTSSKGVKGHHGASDELYLPGGSPVVLLGARVSGSQKAISSGGQKRVTSRRSADMQSSNTDISFKTRKRLNFEDKVISNTAEIESSASQVEDSISEEQEGTSSETSQKRDDLSSEIQPQSKKSFSELYLETVNRKSKCSSVVRHTAAVPSPPYPPNDMKLLEDEFIIDGPDRSFSSQPWVVIPRKGHHMPFPENIAVPQGKKSREKPHRLSEKTLVSNTQTDKTGPVEEAQLSVEEKLGTTLTNELENDCRSTENKTHSENAKKPSARKRTVKQKQRRPSKPNIAQELSMGQNETENRNMSKIGQDKLQINSKRNMEDHEEVRNEPTPKKPVPALGNKKEKDSTQANKEKSRKKCFSRESKRKSVPKEVTLASRRGRRTSQHPSEWWLVKPSEGSVDGNSSKENESSILYPNRKKQTKRNHVSENTEKKPVPSKRKKKEISSRELKSLNVKGSGGNDDISTSQRKPLQNIDADPTQKSLHCSGPTRGSEDQNSDMISQNVHLKTRSEQHRSKTQMESTSNSEVCKHSVWEESGPSRFKNLEMPGRSNSEVGDEKDQKSLDLKTRSSNMVPNRNLHHKLVLPSNSPNVRRSNRIRLKPLEYWRGERIDAQESSSGRLVLEIVSPASESVKRKAKRNLDKVNKETNKKRIHLDNPTKTKIKVSLDIPLGDPFQATLAKDPETREIVPMDLIRPRDTYHFFVEQHGLKVYKTLNTTFFSTGKLVLGPHEEKGKQHVGQDILVFYVNFGDLLCTLHETPYMLTTGDSFYVPSGNHYNIKNLLNVESCLLFTQIKR MASFHLDHLKNYHRRFCRPSRAPNIPTKQGQNMLEILQDCFEEQNSTESLSCSTPKEKDIYVQSTGKERQASHSESVPISSRR

In [2]:
# fetch name list
import os
hhm_path = 'hhm/'
# pdb_path = 'data/pdb_example/'
hhm_path_files = os.listdir(hhm_path)  
name_list = []
for fi in hhm_path_files: 
    hhm_name = fi.split('.')[0]
    name_list.append(hhm_name)
print(len(name_list))

16


In [3]:
# generate mean space-hhblits
import numpy as np
import os
output_path = '../hhblits/'
os.makedirs(output_path, exist_ok=True)
for uniprot_id in name_list:
    # fetch length
    with open(hhm_path + uniprot_id + '.hhm') as hhm_file:
        hhm_line = hhm_file.readline()
        while hhm_line:
            if(hhm_line[0:4] == 'LENG'):
                hhm_seq_len = int(hhm_line.split()[1])
                break
            hhm_line = hhm_file.readline()
    # fetch 30d feature from .hhm    
    with open(hhm_path + uniprot_id + '.hhm') as hhm_file:     
        hhm_matrix = np.zeros([hhm_seq_len, 30], float)
        hhm_line = hhm_file.readline()
        idxx = 0
        while(hhm_line[0] != '#'):
            hhm_line = hhm_file.readline()
        for i in range(0,5):
            hhm_line = hhm_file.readline()
        while hhm_line:
            if(len(hhm_line.split()) == 23):
                idxx += 1
                if(idxx == hhm_seq_len + 1):
                    break
                each_item = hhm_line.split()[2:22]
                for idx, s in enumerate(each_item):
                    if(s == '*'):
                        each_item[idx] = '99999'                            
                for j in range(0, 20):
                    try:
                        hhm_matrix[idxx - 1, j] = int(each_item[j])                      
                    except IndexError:
                        pass
            elif(len(hhm_line.split()) == 10):
                each_item = hhm_line.split()[0:10]
                for idx, s in enumerate(each_item):
                    if(s == '*'):
                        each_item[idx] = '99999'                             
                for j in range(20, 30):
                    try:
                        hhm_matrix[idxx - 1, j] = int(each_item[j - 20])                        
                    except IndexError:
                        pass                            
            hhm_line = hhm_file.readline()
        #print(hhm_matrix.shape) # # seq_len*30
        #print(hhm_matrix)
    
    with open(os.path.join(output_path,uniprot_id + '.hhm'),'w+') as out_file:
        np.savetxt(out_file, hhm_matrix, fmt='%.6f')   

In [4]:
# rename 
import os, shutil
import pandas as pd
df = pd.read_csv('final.csv')
print(df.shape)
for i in range(df.shape[0]):
    idx = df.iloc[i,0]

    mut_old = 'pro_mut_' + str(i) + '.hhm'
    ori_old = 'pro_ori_' + str(i) + '.hhm'

    mut_new = 'pro_mut_' + str(idx) + '.hhm'
    ori_new = 'pro_ori_' + str(idx) + '.hhm'
    shutil.move(('data/'+mut_old),('new/'+mut_new))
    shutil.move(('data/'+ori_old),('new/'+ori_new))

(3803, 41)


FileNotFoundError: [Errno 2] No such file or directory: 'data/pro_mut_0.hhm'